In [2]:
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

# Load Oxford Pets dataset from TFDS
(train_ds, test_ds), info = tfds.load('oxford_iiit_pet:3.*.*', split=['train', 'test'], with_info=True)

# Define preprocessing functions
def preprocess_image(data):
    image = tf.image.resize(data['image'], (32, 32))  # Resize image
    image = tf.cast(image, tf.float32) / 255.0  # Normalize pixel values to [0, 1]
    label = tf.image.resize(data['segmentation_mask'], (32, 32))  # Resize label
    label = tf.cast(label, tf.int32)  # Ensure labels are integers
    
    # Convert labels to one-hot categorical
    num_classes = info.features['label'].num_classes
    label = tf.one_hot(label, depth=num_classes)
    
    # Reshape labels
    label = tf.reshape(label, (32, 32, num_classes))  # Reshape to (32, 32, num_classes)
    
    return image, label

# Apply preprocessing to datasets
train_ds = train_ds.map(preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
test_ds = test_ds.map(preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)


In [3]:
for image, label in train_ds.take(1):
    print("Image shape:", image.shape)
    print("Label shape:", label.shape)

Image shape: (32, 32, 3)
Label shape: (32, 32, 37)


In [4]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Conv2D, UpSampling2D
from tensorflow.keras.models import Model

def create_segmentation_model(input_shape, num_classes):
    # Load ResNet50 backbone
    resnet50_base = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)

    # Add segmentation-specific layers on top of ResNet50
    x = resnet50_base.output
    x = tf.keras.layers.UpSampling2D(size=(32, 32))(x)  # Upsample to match original image size
    x = tf.keras.layers.Conv2D(num_classes, kernel_size=(1, 1), activation='softmax')(x)

    # Create segmentation model
    model = tf.keras.models.Model(inputs=resnet50_base.input, outputs=x)
    return model

# Instantiate the model
input_shape = (32, 32, 3)  # Adjust input shape as per your dataset
num_classes = info.features['label'].num_classes
model = create_segmentation_model(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 16, 16, 64)   256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [15]:
from keras.callbacks import EarlyStopping
early_stopping_monitor = EarlyStopping(patience=10, monitor='loss', verbose=1)
model.fit(train_ds.batch(32), epochs=100,callbacks=[early_stopping_monitor])

Epoch 1/100
115/115 [==============================] - 11s 89ms/step - loss: 0.8648 - accuracy: 0.6249
Epoch 2/100
115/115 [==============================] - 10s 90ms/step - loss: 0.9094 - accuracy: 0.6168
Epoch 3/100
115/115 [==============================] - 11s 94ms/step - loss: 0.8857 - accuracy: 0.6224
Epoch 4/100
115/115 [==============================] - 11s 96ms/step - loss: 0.8687 - accuracy: 0.6257
Epoch 5/100
115/115 [==============================] - 12s 100ms/step - loss: 0.8664 - accuracy: 0.6262
Epoch 6/100
115/115 [==============================] - 10s 89ms/step - loss: 0.8591 - accuracy: 0.6263
Epoch 7/100
115/115 [==============================] - 11s 94ms/step - loss: 0.8675 - accuracy: 0.6237
Epoch 8/100
115/115 [==============================] - 11s 98ms/step - loss: 0.8690 - accuracy: 0.6243
Epoch 9/100
115/115 [==============================] - 10s 88ms/step - loss: 0.8745 - accuracy: 0.6272
Epoch 10/100
115/115 [==============================] - 10s 84ms/step - 

In [16]:
# Evaluate the model on test data
loss, accuracy = model.evaluate(test_ds.batch(32))
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

115/115 [==============================] - 3s 24ms/step - loss: 0.9236 - accuracy: 0.5805
Test Loss: 0.9236343502998352
Test Accuracy: 0.5804755091667175


In [17]:
import numpy as np

# Define IoU (Intersection over Union) metric
def calculate_iou(y_true, y_pred):
    intersection = np.sum(np.logical_and(y_true, y_pred))
    union = np.sum(np.logical_or(y_true, y_pred))
    iou = intersection / union
    return iou

# Define Dice coefficient metric
def calculate_dice_coefficient(y_true, y_pred):
    intersection = 2.0 * np.sum(y_true * y_pred)
    dice_coefficient = intersection / (np.sum(y_true) + np.sum(y_pred))
    return dice_coefficient

# Define Pixel Accuracy metric
def calculate_pixel_accuracy(y_true, y_pred):
    pixel_accuracy = np.sum(y_true == y_pred) / (y_true.shape[0] * y_true.shape[1])
    return pixel_accuracy

# Evaluate the model on test data
iou_scores = []
dice_coefficient_scores = []
pixel_accuracy_scores = []

for image, label in test_ds.batch(32):
    # Predict segmentation masks
    pred_mask = model.predict(image)
    
    # Convert one-hot encoded labels back to categorical
    pred_mask_categorical = np.argmax(pred_mask, axis=-1)
    true_mask_categorical = np.argmax(label, axis=-1)
    
    # Calculate metrics for each sample
    for i in range(len(pred_mask_categorical)):
        iou_scores.append(calculate_iou(true_mask_categorical[i], pred_mask_categorical[i]))
        dice_coefficient_scores.append(calculate_dice_coefficient(true_mask_categorical[i], pred_mask_categorical[i]))
        pixel_accuracy_scores.append(calculate_pixel_accuracy(true_mask_categorical[i], pred_mask_categorical[i]))

# Calculate mean scores
mean_iou = np.mean(iou_scores)
mean_dice_coefficient = np.mean(dice_coefficient_scores)
mean_pixel_accuracy = np.mean(pixel_accuracy_scores)

print("Mean IoU:", mean_iou)
print("Mean Dice Coefficient:", mean_dice_coefficient)
print("Mean Pixel Accuracy:", mean_pixel_accuracy)


Mean IoU: 1.0
Mean Dice Coefficient: 1.7932538751048765
Mean Pixel Accuracy: 0.5804755106125647


In [18]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Conv2D, UpSampling2D
from tensorflow.keras.models import Model
import keras

def create_segmentation_model(input_shape, num_classes):
    # Load ResNet50 backbone
    resnet50_base = keras.models.load_model(r"C:\Users\shaif\Downloads\oxf_32_model.h5")
    
    # Add segmentation-specific layers on top of ResNet50
    x = resnet50_base.layers[-4].output
    x = tf.keras.layers.UpSampling2D(size=(32, 32))(x)  # Upsample to match original image size
    x = tf.keras.layers.Conv2D(num_classes, kernel_size=(1, 1), activation='softmax')(x)

    # Create segmentation model
    model = tf.keras.models.Model(inputs=resnet50_base.input, outputs=x)
    return model

# Instantiate the model
input_shape = (32, 32, 3)  # Adjust input shape as per your dataset
num_classes = info.features['label'].num_classes
model = create_segmentation_model(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()


Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 16, 16, 64)   256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [19]:
early_stopping_monitor = EarlyStopping(patience=10, monitor='loss', verbose=1)
model.fit(train_ds.batch(32), epochs=100,callbacks=[early_stopping_monitor])

Epoch 1/100
115/115 [==============================] - 64s 88ms/step - loss: 1.0662 - accuracy: 0.5671
Epoch 2/100
115/115 [==============================] - 10s 88ms/step - loss: 0.9046 - accuracy: 0.5964
Epoch 3/100
115/115 [==============================] - 10s 83ms/step - loss: 0.9004 - accuracy: 0.6030
Epoch 4/100
115/115 [==============================] - 10s 87ms/step - loss: 0.9456 - accuracy: 0.5849
Epoch 5/100
115/115 [==============================] - 10s 82ms/step - loss: 0.9352 - accuracy: 0.5838
Epoch 6/100
115/115 [==============================] - 10s 89ms/step - loss: 0.9245 - accuracy: 0.5872
Epoch 7/100
115/115 [==============================] - 11s 94ms/step - loss: 0.9198 - accuracy: 0.5891
Epoch 8/100
115/115 [==============================] - 10s 89ms/step - loss: 0.9093 - accuracy: 0.5901
Epoch 9/100
115/115 [==============================] - 10s 89ms/step - loss: 0.9057 - accuracy: 0.5951
Epoch 10/100
115/115 [==============================] - 11s 94ms/step - l

In [20]:
# Evaluate the model on test data
loss, accuracy = model.evaluate(test_ds.batch(32))
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

115/115 [==============================] - 6s 44ms/step - loss: 0.9519 - accuracy: 0.5939
Test Loss: 0.9519481658935547
Test Accuracy: 0.593925416469574


In [21]:
import numpy as np
# Define IoU (Intersection over Union) metric
def calculate_iou(y_true, y_pred):
    intersection = np.sum(np.logical_and(y_true, y_pred))
    union = np.sum(np.logical_or(y_true, y_pred))
    iou = intersection / union
    return iou

# Define Dice coefficient metric
def calculate_dice_coefficient(y_true, y_pred):
    intersection = 2.0 * np.sum(y_true * y_pred)
    dice_coefficient = intersection / (np.sum(y_true) + np.sum(y_pred))
    return dice_coefficient

# Define Pixel Accuracy metric
def calculate_pixel_accuracy(y_true, y_pred):
    pixel_accuracy = np.sum(y_true == y_pred) / (y_true.shape[0] * y_true.shape[1])
    return pixel_accuracy

# Evaluate the model on test data
iou_scores = []
dice_coefficient_scores = []
pixel_accuracy_scores = []

for image, label in test_ds.batch(32):
    # Predict segmentation masks
    pred_mask = model.predict(image)
    
    # Convert one-hot encoded labels back to categorical
    pred_mask_categorical = np.argmax(pred_mask, axis=-1)
    true_mask_categorical = np.argmax(label, axis=-1)
    
    # Calculate metrics for each sample
    for i in range(len(pred_mask_categorical)):
        iou_scores.append(calculate_iou(true_mask_categorical[i], pred_mask_categorical[i]))
        dice_coefficient_scores.append(calculate_dice_coefficient(true_mask_categorical[i], pred_mask_categorical[i]))
        pixel_accuracy_scores.append(calculate_pixel_accuracy(true_mask_categorical[i], pred_mask_categorical[i]))

# Calculate mean scores
mean_iou = np.mean(iou_scores)
mean_dice_coefficient = np.mean(dice_coefficient_scores)
mean_pixel_accuracy = np.mean(pixel_accuracy_scores)

print("Mean IoU:", mean_iou)
print("Mean Dice Coefficient:", mean_dice_coefficient)
print("Mean Pixel Accuracy:", mean_pixel_accuracy)


Mean IoU: 1.0
Mean Dice Coefficient: 1.817277040206568
Mean Pixel Accuracy: 0.5939254032944944
